### Create automation to find geo-coordinates of taxi ranks. 

#### I use the Google Maps API which returns a JSON object when the name of a location is given. From this JSON, I extract the geo-coordinates. 

In [1]:
import googlemaps 
import pandas as pd
import numpy as np

In [40]:
api_key = ##########

gmaps = googlemaps.Client(key = api_key)

In [42]:
address = "Station Parade/Wakering Road(Barking Station)"

gmaps.geocode(address)

[{'address_components': [{'long_name': 'Station Parade & Wakering Road',
    'short_name': 'Station Parade & Wakering Road',
    'types': ['intersection']},
   {'long_name': 'Barking', 'short_name': 'Barking', 'types': ['postal_town']},
   {'long_name': 'Greater London',
    'short_name': 'Greater London',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'England',
    'short_name': 'England',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United Kingdom',
    'short_name': 'GB',
    'types': ['country', 'political']},
   {'long_name': 'IG11 8RU',
    'short_name': 'IG11 8RU',
    'types': ['postal_code']}],
  'formatted_address': 'Station Parade & Wakering Road, Barking IG11 8RU, UK',
  'geometry': {'location': {'lat': 51.539692, 'lng': 0.0818324},
   'location_type': 'GEOMETRIC_CENTER',
   'viewport': {'northeast': {'lat': 51.5410409802915,
     'lng': 0.08318138029150203},
    'southwest': {'lat': 51.5383430197085, 'lng': 0.

In [20]:
lat = barking[0]['geometry']['location']['lat']
long = barking[0]['geometry']['location']['lng']
print('Lat and Long of {}' .format(address))
print('lat = {}, long = {}' .format(lat,long))

Lat and Long of Station Parade/Wakering Road(Barking Station)
lat = 51.539692, long = 0.0818324


In [52]:
taxi_ranks = pd.read_csv('/Users/johannesbreit/OneDrive - Imperial College London/irr/Taxi Ranks/'\
                         'all_london_taxi_ranks.csv')

In [53]:
taxi_ranks.head()

,Borough,Location,Spaces,Times of Operation,Rank No.,Notes
0,Barking and Dagenham,Rainham Road South (Dagenham East Station),1,24 hours,5657.0,NaN
1,Barking and\nDagenham,Station Parade/Wakering Road\n(Barking Station),17,24 hours,5722.0,NaN
2,Barnet,Allum Way (Totteridge and Whetstone Station),2,Friday and Saturday 22:00 – 06:00,5833.0,NaN
3,Barnet,Barnet General Hospital,3,24 hours,6030.0,Private land
4,Barnet,Brent Cross Station Car Park (Brent\nCross Sta...,2,Friday and Saturday\n22:00 – 06:00,5851.0,NaN


A little bit of regex is required to remove unwanted line-break characters. 

In [33]:
import re

In [54]:
taxi_ranks['Borough'] = taxi_ranks['Borough'].str.replace('\\n',' ')
taxi_ranks['Location'] = taxi_ranks['Location'].str.replace('\\n',' ')
taxi_ranks['Times of Operation'] = taxi_ranks['Times of Operation'].str.replace('\\n',' ')

In [55]:
taxi_ranks.head(10)

,Borough,Location,Spaces,Times of Operation,Rank No.,Notes
0,Barking and Dagenham,Rainham Road South (Dagenham East Station),1,24 hours,5657.0,NaN
1,Barking and Dagenham,Station Parade/Wakering Road (Barking Station),17,24 hours,5722.0,NaN
2,Barnet,Allum Way (Totteridge and Whetstone Station),2,Friday and Saturday 22:00 – 06:00,5833.0,NaN
3,Barnet,Barnet General Hospital,3,24 hours,6030.0,Private land
4,Barnet,Brent Cross Station Car Park (Brent Cross Stat...,2,Friday and Saturday 22:00 – 06:00,5851.0,NaN
5,Barnet,Chaville Way (Finchley Central Station),2,Friday and Saturday 22:00 – 06:00,5834.0,NaN
6,Barnet,Station Forecourt (East Finchley Station),2,17:00 - 06:00,5819.0,NaN
7,Barnet,North End Road (Golders Green Station),7,24 hours,5436.0,NaN
8,Barnet,"Queen’s Road, Hendon",3,24 hours,5071.0,TLRN
9,Barnet,Station Access Road (High Barnet Station),2,Friday and Saturday 22:00 – 06:00,5832.0,NaN


In [74]:
address + ', Camden' + ', UK'

'Station Parade/Wakering Road(Barking Station), Camden, UK'

In [75]:
lat_list2 = []
long_list2 = []
for i in range(len(taxi_ranks)):
    geo_data = gmaps.geocode(taxi_ranks.loc[i, 'Location'] + ', ' + taxi_ranks.loc[i, 'Borough'] + ', UK')
    #print(location)
    #print(geo_data)
    if len(geo_data) > 0:
        lat = geo_data[0]['geometry']['location']['lat']
        long = geo_data[0]['geometry']['location']['lng']
    else:
        lat = 'empty'
        long = 'empty'
    lat_list2.append(lat)
    long_list2.append(long)

In [78]:
taxi_ranks2 = taxi_ranks.copy()
taxi_ranks2['Longitude'] = long_list2
taxi_ranks2['Latitude'] = lat_list2

In [79]:
taxi_ranks2.head(20)

,Borough,Location,Spaces,Times of Operation,Rank No.,Notes,Longitude,Latitude
0,Barking and Dagenham,Rainham Road South (Dagenham East Station),1,24 hours,5657.0,NaN,0.165839,51.5442
1,Barking and Dagenham,Station Parade/Wakering Road (Barking Station),17,24 hours,5722.0,NaN,0.0818324,51.5397
2,Barnet,Allum Way (Totteridge and Whetstone Station),2,Friday and Saturday 22:00 – 06:00,5833.0,NaN,-0.17915,51.6302
3,Barnet,Barnet General Hospital,3,24 hours,6030.0,Private land,-0.214486,51.6506
4,Barnet,Brent Cross Station Car Park (Brent Cross Stat...,2,Friday and Saturday 22:00 – 06:00,5851.0,NaN,-0.213075,51.5767
5,Barnet,Chaville Way (Finchley Central Station),2,Friday and Saturday 22:00 – 06:00,5834.0,NaN,-0.193361,51.6016
6,Barnet,Station Forecourt (East Finchley Station),2,17:00 - 06:00,5819.0,NaN,-0.274931,51.6137
7,Barnet,North End Road (Golders Green Station),7,24 hours,5436.0,NaN,-0.194416,51.5722
8,Barnet,"Queen’s Road, Hendon",3,24 hours,5071.0,TLRN,-0.222922,51.5833
9,Barnet,Station Access Road (High Barnet Station),2,Friday and Saturday 22:00 – 06:00,5832.0,NaN,-0.193914,51.6503


We need to fill in some places manually which were not automatically found via the Google Maps API call. I look these up manually on Maps and get the geo-coordinates.

In [80]:
taxi_ranks2.loc[taxi_ranks2.Longitude == 'empty']

,Borough,Location,Spaces,Times of Operation,Rank No.,Notes,Longitude,Latitude
172,Hillingdon,Great Central Avenue (South Ruislip Station),2,24 hours,3569.0,NaN,empty,empty
296,Redbridge,Belgrave Road (Ilford Station),2,24 hours,6012.0,NaN,empty,empty
310,Redbridge,Service Road Entrance to Wanstead Station Car ...,2,22:00 – 06:00,5825.0,NaN,empty,empty


In [88]:
#Great Central Avenue (South Ruislip Station): 51.556279, -0.398232
taxi_ranks2.loc[172, 'Longitude'] = -0.398232
taxi_ranks2.loc[172, 'Latitude'] = 51.556279

#Belgrave Road (Ilford Station): 51.559456, 0.066680
taxi_ranks2.loc[296, 'Longitude'] = 0.066680
taxi_ranks2.loc[296, 'Latitude'] = 51.559456

#Service Road Entrance to Wanstead Station Car Park (Wanstead Station): 51.576428, 0.046108
taxi_ranks2.loc[310, 'Longitude'] = 0.046108
taxi_ranks2.loc[310, 'Latitude'] = 51.576428

In [89]:
taxi_ranks2.loc[taxi_ranks2.Longitude == 'empty']

,Borough,Location,Spaces,Times of Operation,Rank No.,Notes,Longitude,Latitude


In [91]:
taxi_ranks2.loc[taxi_ranks2.Longitude < -1]

,Borough,Location,Spaces,Times of Operation,Rank No.,Notes,Longitude,Latitude


In [99]:
taxi_ranks2.Latitude.describe()

count     551.000000
unique    541.000000
top        51.543146
freq        2.000000
Name: Latitude, dtype: float64

In [97]:
taxi_ranks2['Longitude'].describe()

count     551.000000
unique    542.000000
top        -0.152416
freq        2.000000
Name: Longitude, dtype: float64

In [100]:
taxi_ranks2.to_csv('london_taxi_ranks_with_geo_coordinates2.csv')